In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import monai
from glob import glob
import matplotlib.pyplot as plt
from datetime import datetime

import get_param_in_cwd

model_root = get_param_in_cwd('model_root', 'models')
# 模型名称
model_name = get_param_in_cwd('model_name', 'resnet50')

samples = [os.path.join(mydir, f) for f in os.listdir(mydir) if f.endswith('.jpg') or f.endswith('.png')]
print(mydir)
print(samples)
model_name

In [ ]:
from custom.components.comp2 import extract, init_from_model, init_from_onekey

model, transformer, device = init_from_onekey(os.path.join(model_root, model_name, 'viz'))
for n, m in model.named_modules():
    print('Feature name:', n, "|| Module:", m)

In [13]:

target_layer = "features.denseblock4.denselayer16.conv2"
gradcam = monai.visualize.GradCAM(nn_module=model, target_layers=target_layer)

In [ ]:
from datasets.image_loader import default_loader
from custom.components.comp2 import show_cam_on_image
import torch
import os
import random

random.shuffle(samples)
viz_dir = r'F:\Result\IBD_UST_SVS\viz\Grad_CAM\yes'
os.makedirs(viz_dir, exist_ok=True)
for sample in samples:
    img = default_loader(sample)
    sample_ = transformer(img)
    sample_  = sample_.view(1, *sample_.size()).to(device)
    res_cam = gradcam(x=sample_, class_idx=None)
    fig, axes = plt.subplots(1, 2, figsize=(20, 10), facecolor='white')
    axes[0].imshow(img.resize(sample_.size()[2:]))
    axes[0].axis('off')
    axes[1].imshow(show_cam_on_image(img.resize(sample_.size()[2:]), -res_cam[0][0].cpu(), use_rgb=True, reverse=False))
    axes[1].axis('off')
    plt.savefig(f'{viz_dir}/{os.path.basename(sample)}', bbox_inches = 'tight')
    # plt.show()